In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import re

from datetime import datetime
import numpy as np

import warnings
warnings.filterwarnings('ignore')

# 지수 표기법을 10진수로 변경
pd.set_option('display.float_format', '{:.2f}'.format)

import platform

system = platform.system()

if system == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
else:
    !apt-get -y install fonts-nanum
    !fc-cache -fv
    !rm ~/.cache/matplotlib -rf

    plt.rcParams['font.family'] = 'NanumBarunGothic'


입장 조건
* 롱 포지션:

    - 가격 > 200 SMA: 현재 가격은 200일 단순 이동 평균(SMA)보다 높아야 합니다.
    - MACD > 신호선: MACD 값이 신호선보다 높아 상승 모멘텀을 나타냅니다.
    - RSI > 50: 상대강도지수(RSI)가 50을 초과하여 강세 추세를 나타냅니다.
    - 현재 거래량 > 평균 거래량: 현재 거래량이 지난 50개 기간의 평균 거래량을 초과합니다.

* 매도 포지션:

    - 가격 < 200 SMA: 현재 가격이 200일 SMA보다 낮습니다.
    - MACD < 시그널선: MACD 값이 시그널선보다 작아 하락 모멘텀을 나타냅니다.
    - RSI < 50: RSI가 50 미만으로 약세 추세를 나타냅니다.
    - 현재 거래량 > 평균 거래량: 현재 거래량이 지난 50개 기간의 평균 거래량을 초과합니다.

종료 조건
* 롱 포지션:

    - 손절매: 진입 가격보다 낮은 특정 비율(예: 0.5% 또는 조정된 가격)로 설정합니다.
    - 전체 포지션 청산: 가격이 볼린저 밴드의 중간선에 닿을 때, 이는 추세 반전 가능성을 나타냅니다.

* 매도 포지션:

    - 손절매: 진입 가격보다 높은 특정 비율로 설정됩니다.
    - 전체 포지션 청산: 가격이 볼린저 밴드의 중간선에 닿을 때.


In [2]:
backtest = pd.read_csv("./data/Grouped_Backtest_Results_by_Indicator.csv")

In [7]:
backtest_long = backtest.query("Position == 'Long'")
backtest_short = backtest.query("Position == 'Short'")

In [13]:
backtest_long.describe()

,Entry_RSI,Exit_RSI,Max_Profit_RSI,Entry_MACD,Exit_MACD,Max_Profit_MACD,Entry_Signal_Line,Exit_Signal_Line,Max_Profit_Signal_Line,Entry_Bollinger_Percentage,Exit_Bollinger_Percentage,Max_Profit_Bollinger_Percentage,Entry_Price,Exit_Price,Profit_Loss,Max_Profit_Value
count,45.00,45.00,45.00,45.00,45.00,45.00,45.00,45.00,45.00,45.00,45.00,45.00,45.00,45.00,45.00,45.00
mean,43.42,57.22,60.89,-4.44,3.25,15.31,-0.51,-0.38,10.18,25.05,75.94,77.81,62523.12,62637.18,-22.76,160.57
std,18.05,14.41,15.00,16.54,12.20,45.38,11.49,13.60,41.83,56.98,54.38,25.96,573.85,641.02,184.46,238.39
min,20.86,30.83,19.55,-61.83,-47.70,-34.93,-30.02,-34.72,-41.97,-34.17,-18.55,1.87,61789.12,61876.00,-382.25,3.57
25%,30.37,46.32,53.77,-11.94,-2.92,-6.92,-7.13,-8.49,-10.85,-10.76,-1.14,73.06,61980.43,62011.52,-102.97,50.98
50%,36.00,59.34,62.04,-4.21,2.55,2.54,0.27,0.23,-0.92,-3.79,103.62,86.39,62424.50,62533.93,-50.19,86.73
75%,62.38,66.72,70.11,0.26,8.48,22.28,6.13,7.84,20.56,100.39,110.97,94.19,62798.10,62956.06,8.47,161.05
max,81.80,87.74,88.73,33.31,30.94,224.12,20.98,28.22,186.36,143.13,133.01,109.30,63763.43,63890.30,696.38,1379.19


In [16]:
backtest_short.query("Profit_Loss > 0")

,Position,Entry_RSI,Exit_RSI,Max_Profit_RSI,Entry_MACD,Exit_MACD,Max_Profit_MACD,Entry_Signal_Line,Exit_Signal_Line,Max_Profit_Signal_Line,Entry_Bollinger_Percentage,Exit_Bollinger_Percentage,Max_Profit_Bollinger_Percentage,Entry_Time,Entry_Price,Exit_Time,Exit_Price,Profit_Loss,Max_Profit_Value,Max_Profit_Time
5,Short,32.03,62.57,34.37,-11.69,-4.72,-32.56,-4.40,-14.43,-32.29,-0.77,111.74,10.83,2024-10-05 16:37:00+00:00,62071.56,2024-10-05 17:22:00+00:00,62018.04,53.52,170.78,2024-10-05 17:09:00+00:00
7,Short,36.44,61.93,56.41,-6.24,2.05,-7.28,1.54,-3.23,-7.76,-6.36,100.81,77.04,2024-10-05 18:06:00+00:00,61963.55,2024-10-05 19:26:00+00:00,61904.15,59.40,155.22,2024-10-05 19:20:00+00:00
8,Short,24.52,56.08,41.36,-11.24,-5.86,-9.37,-5.56,-8.54,-9.21,-13.29,105.59,6.50,2024-10-05 19:46:00+00:00,61790.23,2024-10-05 21:06:00+00:00,61769.14,21.09,88.09,2024-10-05 21:05:00+00:00
12,Short,34.25,62.00,19.71,-0.06,-12.50,-21.97,10.92,-18.42,-7.05,-11.78,113.35,6.90,2024-10-06 00:04:00+00:00,62002.59,2024-10-06 00:25:00+00:00,61999.02,3.57,69.79,2024-10-06 00:09:00+00:00
37,Short,35.71,71.87,16.52,4.69,-1.05,-48.71,20.46,-17.12,-36.56,-17.75,102.35,-8.76,2024-10-06 19:33:00+00:00,62754.00,2024-10-06 20:12:00+00:00,62748.71,5.29,173.18,2024-10-06 19:56:00+00:00
43,Short,31.47,59.12,21.62,-17.14,1.98,-71.87,-0.36,-9.33,-44.95,-18.19,101.94,-5.95,2024-10-07 02:32:00+00:00,63739.81,2024-10-07 04:01:00+00:00,63568.09,171.72,397.27,2024-10-07 03:37:00+00:00
51,Short,36.71,72.30,4.74,-26.45,20.62,-69.89,-17.25,10.67,-50.97,-2.92,149.88,0.17,2024-10-07 09:03:00+00:00,63476.74,2024-10-07 11:57:00+00:00,63045.56,431.18,821.73,2024-10-07 11:08:00+00:00
61,Short,29.82,71.18,6.42,-14.42,-16.15,-160.77,8.91,-69.43,-119.25,-22.05,105.32,3.03,2024-10-07 18:08:00+00:00,63558.76,2024-10-07 19:06:00+00:00,63199.95,358.81,828.98,2024-10-07 18:51:00+00:00
66,Short,33.89,79.61,16.00,-28.68,45.88,-193.71,-19.86,27.85,-121.68,-8.31,106.17,-14.08,2024-10-07 22:33:00+00:00,63128.87,2024-10-08 00:33:00+00:00,62577.23,551.64,1013.72,2024-10-07 23:00:00+00:00
70,Short,25.72,70.81,6.66,-2.26,-19.29,-40.63,15.45,-34.17,-23.12,-1.03,103.39,3.41,2024-10-08 02:14:00+00:00,62570.70,2024-10-08 02:37:00+00:00,62562.66,8.04,157.92,2024-10-08 02:23:00+00:00


In [17]:
backtest_long.query("Profit_Loss > 0")

,Position,Entry_RSI,Exit_RSI,Max_Profit_RSI,Entry_MACD,Exit_MACD,Max_Profit_MACD,Entry_Signal_Line,Exit_Signal_Line,Max_Profit_Signal_Line,Entry_Bollinger_Percentage,Exit_Bollinger_Percentage,Max_Profit_Bollinger_Percentage,Entry_Time,Entry_Price,Exit_Time,Exit_Price,Profit_Loss,Max_Profit_Value,Max_Profit_Time
9,Long,67.77,38.19,78.98,-1.46,7.66,22.28,-7.13,15.84,22.89,118.56,-4.68,95.40,2024-10-05 21:07:00+00:00,61789.12,2024-10-05 21:50:00+00:00,61876.00,86.88,160.88,2024-10-05 21:44:00+00:00
11,Long,81.80,36.21,77.83,17.04,5.34,20.96,5.07,13.67,17.19,100.39,-13.66,86.39,2024-10-05 22:05:00+00:00,61974.33,2024-10-06 00:03:00+00:00,62011.52,37.19,137.94,2024-10-05 23:57:00+00:00
14,Long,62.38,36.84,63.46,0.03,-2.89,42.92,-7.20,-0.62,33.42,108.05,-2.67,85.50,2024-10-06 01:02:00+00:00,61953.13,2024-10-06 02:32:00+00:00,61986.66,33.53,202.28,2024-10-06 01:28:00+00:00
20,Long,73.85,35.13,67.06,10.54,1.32,11.34,-0.69,5.53,8.71,121.35,-0.39,109.30,2024-10-06 05:22:00+00:00,61967.41,2024-10-06 06:40:00+00:00,61980.80,13.39,54.53,2024-10-06 06:21:00+00:00
22,Long,64.96,46.59,57.88,0.11,-0.80,24.19,-5.47,1.98,20.60,102.00,-4.83,70.37,2024-10-06 08:16:00+00:00,61979.48,2024-10-06 08:55:00+00:00,61987.95,8.47,99.87,2024-10-06 08:32:00+00:00
27,Long,59.08,35.88,86.40,7.28,4.18,48.52,4.13,14.71,32.63,105.80,-1.14,103.84,2024-10-06 12:07:00+00:00,62163.81,2024-10-06 13:29:00+00:00,62248.42,84.61,203.58,2024-10-06 13:04:00+00:00
29,Long,74.91,46.32,70.52,23.84,-6.25,84.11,16.84,10.04,75.96,100.29,-3.37,85.02,2024-10-06 14:26:00+00:00,62384.88,2024-10-06 15:20:00+00:00,62518.63,133.75,395.08,2024-10-06 14:58:00+00:00
31,Long,77.08,31.82,88.73,-2.25,-9.60,76.76,-15.84,10.75,63.30,103.14,-2.45,91.22,2024-10-06 15:40:00+00:00,62613.78,2024-10-06 16:26:00+00:00,62668.11,54.33,318.03,2024-10-06 16:08:00+00:00
36,Long,74.69,39.58,64.09,13.45,14.72,29.89,4.79,24.40,27.87,118.29,-3.05,94.00,2024-10-06 19:03:00+00:00,62798.10,2024-10-06 19:32:00+00:00,62799.89,1.79,185.68,2024-10-06 19:30:00+00:00
40,Long,70.77,38.33,65.46,27.62,-0.56,41.77,20.98,7.84,26.98,143.13,-14.20,96.88,2024-10-06 23:00:00+00:00,62706.34,2024-10-07 00:10:00+00:00,62751.18,44.84,217.37,2024-10-06 23:44:00+00:00
